In [1]:
# Create monthly report url

# To run this directly from the command line, 
# jupyter nbconvert --execute "USDA Monthly Report.ipynb" --to asciidoc

from datetime import datetime, timedelta, date
import urllib.parse

report_date = '2/29/2024'

end = datetime.strptime(report_date, '%m/%d/%Y')
start = date(end.year, end.month, 1)
# start = date(end.year, 1, 1)

ncc_url = r"https://newmarketchurch.breezechms.com"
base_url = ncc_url + r"/events/reports/243292484#/"

import urllib.parse
params = {
    'start_date': urllib.parse.quote(start.strftime('%m/%d/%Y'), safe='/'),
    'end_date': urllib.parse.quote(end.strftime('%m/%d/%Y'), safe='/'),
    'event_ids': '2949650',
    'attendance_status': 'attended',
    'attendance_operator': 'any',
    'page': 'spreadsheet',
}

print("Download the spreadsheet from the following link. Copy the full path into the 'attendancereport' below.")
print(base_url + urllib.parse.urlencode(params))

Download the spreadsheet from the following link. Copy the full path into the 'attendancereport' below.
https://newmarketchurch.breezechms.com/events/reports/243292484#/start_date=02%2F01%2F2024&end_date=02%2F29%2F2024&event_ids=2949650&attendance_status=attended&attendance_operator=any&page=spreadsheet


In [2]:
# This is the file that was downloaded from https://newmarketchurch.breezechms.com/events/reports/243292472#/&start_date=10%2F01%2F2023&end_date=10%2F31%2F2023&event_ids=2949650&attendance_status=attended&attendance_operator=any&page=spreadsheet

attendancereport = r"C:\Users\ralph\Downloads\FoodPantry-attendance-02-03-2024.xlsx"
peopleexport = r"C:\Users\ralph\Downloads\newmarketchurch-people-02-03-2024.xlsx"

# Paste this file into this month's tab in https://docs.google.com/spreadsheets/d/1hAdgMh7_m73L--bR-rUT2dZPMwEJ8s2e/edit#gid=1826328980
paste_file = r"C:\Users\ralph\Downloads\paste_me.csv"
summary_file = r"C:\Users\ralph\Downloads\USDA_summary.csv"

import numpy as np
import pandas as pd

# Read in the downloaded Excel file 
attendance = pd.read_excel(attendancereport)

# drop the Total row
attendance = attendance[attendance['Breeze ID'].notna()]

people = pd.read_excel(peopleexport)
print('{count} people in input.'.format(count = len(attendance.index)))

115 people in input.


In [3]:
display(attendance.columns)

Index(['Breeze ID', 'First Name', 'Last Name', 'Feb 1, 2024', 'Feb 8, 2024',
       'Feb 15, 2024', 'Feb 22, 2024', 'Feb 29, 2024', 'Person Totals'],
      dtype='object')

In [4]:
# Keep only the columns we use.

attendance = attendance[[
    'Breeze ID', 
    'First Name', 
    'Last Name', 
    'Person Totals'
]]

people = people[[
    'Breeze ID',
    'Date of First Contact',
    'How many people live in your household? (including yourself)',
    'Total number of people aged 18 or under in household?',
    'Total number of people between the ages of 19-59 in household?',
    'Total number of people aged 60 or older in Household?',
    'City'
]]


In [5]:
# Include household numbers from the people profiles.

report = pd.merge(attendance, people, on = 'Breeze ID', how = "inner")
report.fillna(0, inplace=True)

# print('{count} people in report.'.format(count = len(report.index)))

In [6]:
# Clean up the City column.

def clean(word):
    if word == 0:
        return 'unknown'
    return word.strip().title()
    
report['City'] = report['City'].apply(clean)

In [7]:
# Validate the data 

# Check for numbers in numeric fields
numeric_fields = [
    'Total number of people aged 18 or under in household?',
    'Total number of people between the ages of 19-59 in household?',
    'Total number of people aged 60 or older in Household?',
]
# fillna() fills with a float, so make it an int.
for column in numeric_fields:
    report[column] = report[column].astype(int)

for field in numeric_fields:
    if len(report[~report.map(np.isreal)[field]]) > 0:
        display(field, report[~report.map(np.isreal)[field]])

bad_in_house = report.loc[report['How many people live in your household? (including yourself)'] == 0] 
if len(bad_in_house):
    print("Missing 'Total in house' for")
    display(report.loc[report['How many people live in your household? (including yourself)'] == 0])

bad_totals = report.loc[report['How many people live in your household? (including yourself)'] != 
        report['Total number of people aged 18 or under in household?'] + 
        report['Total number of people between the ages of 19-59 in household?'] + 
        report['Total number of people aged 60 or older in Household?']
    ]
if len(bad_totals) > 0:
    print("Age breakdowns do not add up to total for")
    display(bad_totals)

In [8]:
report.to_csv(paste_file, index=False)

In [10]:
# Create the summary for the USDA monthly report.

MealsPerWeek = 15

numberOfSeniors = 25
SeniorCenter = {
    # 'City': 'Newmarket',
    'Households': numberOfSeniors,
    'Individuals': numberOfSeniors * attendance['Person Totals'].max(),
    'Meals': MealsPerWeek * numberOfSeniors * attendance['Person Totals'].max(),
    'Over 60': numberOfSeniors * attendance['Person Totals'].max()
}

summary = []

for town in set([t for t in report['City'].unique().flatten()]):
    row = {}
    row['Town'] = town
    filtered = report.loc[report['City'] == town]
    row['Households'] = len(filtered)
    row['Individuals'] = (filtered['How many people live in your household? (including yourself)'] * filtered['Person Totals']).sum()
    row['Meals'] = MealsPerWeek * (filtered['How many people live in your household? (including yourself)'] * filtered['Person Totals']).sum()
    row['Over 60'] = (filtered['Total number of people aged 60 or older in Household?'] * filtered['Person Totals']).sum()
    row['Children'] = (filtered['Total number of people aged 18 or under in household?'] * filtered['Person Totals']).sum()
    if town == SeniorCenter['City']:
        for key, value in SeniorCenter.items():
            row[key] += value
    summary.append(row)
    
row = {}
row['Town'] = 'TOTAL'
row['Households'] = len(report)
row['Individuals'] = (report['How many people live in your household? (including yourself)'] * report['Person Totals']).sum()
row['Meals'] = MealsPerWeek * (report['How many people live in your household? (including yourself)'] * report['Person Totals']).sum()
row['Over 60'] = (report['Total number of people aged 60 or older in Household?'] * report['Person Totals']).sum()
row['Children'] = (report['Total number of people aged 18 or under in household?'] * report['Person Totals']).sum()
for key, value in SeniorCenter.items():
    row[key] += value

summary.append(row)

pd.DataFrame(summary).to_csv(summary_file, index=False)
display(pd.DataFrame(summary))


KeyError: 'City'